# Calculate ICE Demand (EVI-Pro Reverse Engineer)

## Imports

In [ ]:
import pandas as pd
import geopandas as gpd
import movingpandas as mpd
from fiona.crs import from_epsg

import warnings
warnings.simplefilter("ignore")

# Custom utilities
from src.utils import Vehicle, Simulation, ChargingEvents, HexGrid
from src.models import Random_Sample_Charge_Location_Model, Linear_Kwh_Model

## Load Trajectories

In [ ]:
data_all = pd.read_csv('../data/raw/part-00167-tid-8316490494947456795-71092cdb-87a8-47f5-9d32-296121158b22-3052-1.c000.csv', chunksize=5000)

In [ ]:
for data in data_all:
    data = data[data.battery_pack.isna()]
    dataGeo = gpd.GeoDataFrame(data, geometry=gpd.points_from_xy(data.decr_lng, data.decr_lat), crs=from_epsg(4326))
    dataGeo['t'] = pd.to_datetime(dataGeo.element_time_local, utc=True)
    dataGeo = dataGeo.set_index('t')
    traj_collection = mpd.TrajectoryCollection(dataGeo, 'hashed_vin')
    break

## Create Vehicles

In [ ]:
# For each vehicle trajectory create a vehicle object
vehicles = []
for traj in traj_collection.trajectories:
    vehicles.append(Vehicle(traj))

## Create Trained Models

In [ ]:
# Train the linear model
linear_kwh_model = Linear_Kwh_Model()
linear_kwh_model.train()

In [ ]:
# No training required for this model
random_sampling_model = Random_Sample_Charge_Location_Model()

## Run Simulation

In [ ]:
sim = Simulation(vehicles[0:100], random_sampling_model, linear_kwh_model)

In [ ]:
# Print out the options after the run is complete
sim.run()

## See results and visualize

In [ ]:
# sim.results['charging_events']
# sim.results['hourly_charging_events']

In [ ]:
# sim.map(resolution=7, hour=16)